<a href="https://colab.research.google.com/github/LilyRosa/Matematica-Numerica-Google-Colab/blob/main/notebooks/cap3/Gauss-Seidel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import array, absolute, zeros, copy
import pandas as pd

# Algoritmo de Gauss-Seidel

Algoritmo utilizado para dar solución a sistemas lineales de orden $n$ en la forma $AX = B$ con error menor que $\varepsilon$



## Implementación

### Clase Auxiliar de Resultado de Seidel

In [2]:
class ResultadoSeidel:
    def __init__(self):
        self.lista_x = []
        self.delta = 0
        self.error = 0
        self.iteracion_0 = False

Clase auxiliar usada para guardar los resultados de cada iteración del algoritmo de Seidel

### Algoritmo de Gauss-Seidel

` gauss_seidel(a, b, x0, f_convergencia, tol, max_iter): ` Implementación del algoritmo de Seidel para dar solución a sistemas de ecuaciones lineales

##### Parámetros

- ` a ` : matriz de los coeficientes $A$
- ` b ` : matriz de los términos independientes $B$
- ` x0 ` : matriz columna que representa los valores estimados de solución (se puede utilizar la matriz trivial)
- ` f_convergencia ` : define el factor de convergencia de la matriz $A$
- ` tol ` : cota para el error absoluto
- ` max_iter `: cantidad máxima de iteraciones

In [3]:
def gauss_seidel(a, b, x0, f_convergencia, tol, max_iter):
    paso = 1
    resultado = copy(x0)
    error = zeros(a.shape[1])
    condicion = True
    retorno = []
    r = ResultadoSeidel()

    r.iteracion_0 = True
    r.lista_x = error.tolist()
    retorno.append(r)

    while condicion:
        r = ResultadoSeidel()
        for i in range(a.shape[0]):
            valor_temp = 0
            for j in range(a.shape[1]):
                if i == j:
                    valor_temp += b[i] / a[i][j]
                else:
                    valor_temp += -a[i][j] / a[i][i] * resultado[j]
            error[i] = abs(resultado[i] - valor_temp)
            resultado[i] = valor_temp

        r.lista_x = resultado.tolist()
        r.delta = max(error)
        r.error = max(error)*abs(f_convergencia / (1 - f_convergencia))
        retorno.append(r)

        paso += 1
        condicion = max(error) > tol and paso <= max_iter

    return retorno

### Hallar factor de convergencia $\beta$

` hallar_factor_convergencia(a): ` Halla el factor de convergencia $\beta$ de la matriz $A$

##### Parámetros
` a ` : matriz de los coeficientes $A$

In [4]:
def hallar_factor_convergencia(a):
    a = absolute(a)
    resultado = []

    for i in range(a.shape[0]):
        total_fila = sum(a[i])

        if total_fila - a[i][i] < 0:
            raise Exception("El factor de convergencia de la matriz tiene que ser mayor que 0")

        q = 0
        p = 0
        for j in range(a.shape[1]):
            if i > j:
                p += a[i][j] / a[i][i]
            elif i < j:
                q += a[i][j] / a[i][i]

        resultado.append(q / (1 - p))

    return max(resultado)

### Métodos Auxiliares

In [5]:
def hallar_f_convergencia_error(f_convergencia):
    return abs(f_convergencia / (1 - f_convergencia))


def convertir_headers_resultados(lista_resultados_seidel):
    lista = []

    r = lista_resultados_seidel[0]
    for i in range(len(r.lista_x)):
        lista.append(f'X{i + 1}')
    lista.append('Delta(δ)')
    lista.append('Error')

    return lista


def convertir_resultados(lista_resultados_seidel):
    lista = []
    for r in lista_resultados_seidel:
        l = []
        if r.iteracion_0:
            for x in r.lista_x:
                l.append(x)
            l.append('-------')
            l.append('-------')
        else:
            for x in r.lista_x:
                l.append(x)
            l.append('{:.7f}'.format(r.delta))
            l.append('{:.7f}'.format(r.error))

        lista.append(l)

    df = pd.DataFrame(data=lista, columns=convertir_headers_resultados(lista_resultados_seidel))
    df.index.name = 'Iteración'
    return df

## Inserción de datos

#### Datos

- ` a ` : matriz de los coeficientes $A$
- ` b ` : matriz de los términos independientes $B$
- ` x0 ` : matriz columna que representa los valores estimados de solución (se puede utilizar la matriz trivial)
- ` tol ` : cota para el error absoluto
- ` max_iter `: cantidad máxima de iteraciones

In [6]:
a = array([[5, -1, 1],
           [2, 5, -1],
           [-1, 1, 5]])

b = array([10, 12, 10])

# No cambiar el argumento dtype para el correcto funcionamiento del algoritmo
x0 = array([0, 0, 0], dtype=float)

tol = 0.005

max_iter = 100

## Salida de datos

### Factor de Convergencia $\beta$

In [7]:
print(f'El factor de convergencia es {hallar_factor_convergencia(a)}')
print('β/(1-β) = {:.5f}\n'.format(hallar_f_convergencia_error(hallar_factor_convergencia(a))))

El factor de convergencia es 0.4
β/(1-β) = 0.66667


### Resultados de la aplicación del método de Gauss-Seidel

In [8]:
r = gauss_seidel(a, b, x0, hallar_factor_convergencia(a), tol, max_iter)
df = convertir_resultados(r)
df

,X1,X2,X3,Delta(δ),Error
Iteración,,,,,
0,0.000000,0.000000,0.000000,-------,-------
1,2.000000,1.600000,2.080000,2.0800000,1.3866667
2,1.904000,2.054400,1.969920,0.4544000,0.3029333
3,2.016896,1.987226,2.005934,0.1128960,0.0752640
4,1.996258,2.002683,1.998715,0.0206377,0.0137585
5,2.000794,1.999426,2.000274,0.0045354,0.0030236
